In [250]:
import numpy as np
import pandas as pd 
import streamlit as st
from sklearn.model_selection import cross_validate 
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler , OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


In [76]:
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression

In [484]:
from datasist.structdata import detect_outliers
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')

In [15]:
df = pd.read_csv('cardio_train.csv',';')

In [17]:
df.drop('id',axis= 1 ,inplace = True)

In [21]:
df.duplicated().sum()

24

In [22]:
df.drop_duplicates(inplace = True , ignore_index = True )

In [26]:
df.isna().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [31]:
df['age'] = df['age']/365

In [ ]:
lis= ['age','gender','height','weight','ap_hi','ap_lo']

In [ ]:
for i in lis :
    dro = detect_outliers(df,0,[i])
    df.drop(dro,axis= 0 , inplace = True)

In [ ]:
df.reset_index(drop = True,inplace = True )

In [616]:
list1 = []
list1.append(('scaler',StandardScaler()))
list1.append(('encoder',LinearRegression()))

In [617]:
pipeline = Pipeline(steps = list1)

In [618]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('encoder', LinearRegression())])

In [619]:
x = df.drop(['cholesterol'],axis = 1)
y = df['cholesterol']

In [620]:
score = cross_validate(pipeline , x , y , cv = 5 , scoring="r2" , return_train_score=True)

In [621]:
score["train_score"].mean()

0.25469512821786355

In [622]:
score["test_score"].mean()

0.2541005778840286

In [623]:
pipline.fit(x,y)

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 RandomForestRegressor(max_depth=19, min_samples_leaf=10,
                                       min_samples_split=10))])

In [624]:
test  = df.iloc[1: 4].drop('age',axis = 1)

In [625]:
df.iloc[3:4].drop('age',axis = 1)

,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
3,2,169,82.0,150,100,1,1,0,0,1,1


In [626]:
test  = df.iloc[13: 17].drop('height',axis = 1)

In [627]:
df.iloc[13: 17]

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
13,58.345205,1,170,75.0,130,70,1,1,0,0,0,0
14,45.882192,1,158,52.0,110,70,1,3,0,0,1,0
15,47.895890,1,154,68.0,100,70,1,1,0,0,0,0
16,59.602740,2,162,56.0,120,70,1,1,1,0,1,0


In [634]:
pipline.predict(test)

array([1.31901037, 1.59584765, 1.19537083, 1.36677322])

In [629]:
import joblib

In [635]:
joblib.dump(pipline,"pipline.h5")
joblib.dump(x.columns,"input.h5")

['input.h5']

In [638]:
model = joblib.load('pipline.h5')
inp = joblib.load('input.h5')

In [ ]:
import streamlit  as st
import joblib
import pandas as pd

In [676]:
%%writefile Cholesterol.py

import streamlit  as st
import joblib
import pandas as pd
import sklearn 

model = joblib.load('pipline.h5')
Inputs = joblib.load('input.h5')



def predict(age, gender, height, weight, ap_hi, ap_lo, gluc, smoke,alco, active, cardio) :
    test_df = pd.DataFrame(columns = Inputs)
    test_df.at[0,'age'] = age
    test_df.at[0,'gender'] = gender
    test_df.at[0,"height"] = height
    test_df.at[0,"weight"] = weight
    test_df.at[0,"ap_hi"] = ap_hi
    test_df.at[0,"ap_lo"] = ap_lo
    test_df.at[0,"gluc"] = gluc
    test_df.at[0,"smoke"] = smoke
    test_df.at[0,"alco"] = alco
    test_df.at[0,"active"] = active
    test_df.at[0,"cardio"] = cardio
    result = model.predict(test_df)[0]
    return result

def main () :
    st.title('Estimate your Cholesterol') 
    age = st.slider("age" , min_value=10 , max_value=90 , value=25 , step=1)
    gender = st.selectbox("gender" ,[0, 1])
    height = st.slider("height" , min_value=0.0 , max_value= 200.0 , value=25.0 , step=0.2)
    weight = st.slider("weight" , min_value=0.0 , max_value=110.0 , value=25.0 , step=0.2)
    ap_hi = st.slider("ap_hi" , min_value=0.0 , max_value=180.0 , value=25.0 , step=0.2)
    ap_lo = st.slider("ap_lo" , min_value=0.0 , max_value=110.0 , value=25.0 , step=0.2)
    smoke = st.selectbox("smoke",[1,0])
    alco = st.selectbox("alco",[1,0])
    gluc = st.selectbox("gluc" ,[ 1, 2, 3] )
    active = st.selectbox("active",[0, 1])
    cardio = st.selectbox("cardio",[1,0])
    
    if st.button('Predict') :
        result = predict(age, gender, height, weight, ap_hi, ap_lo, gluc, smoke,alco, active, cardio)
        st.write('your cholesterol level is {}'.format(result))
                  

        
        
if __name__ == '__main__' :
    main()
    
  

Writing Cholesterol.py


In [ ]:
!streamlit run Cholesterol.py

In [668]:
model.feature_names_in_

array(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'gluc',
       'smoke', 'alco', 'active', 'cardio'], dtype=object)

In [670]:
df['alco'].unique()

array([0, 1], dtype=int64)